# Bedrock AgentCore Browser를 사용한 Strands Agents

이 워크샵은 Strands Agents를 Amazon Bedrock AgentCore Browser와 통합하여 웹 브라우저와 동적으로 상호작용할 수 있는 AI 에이전트를 생성하는 방법을 보여줍니다.

## 개요

이 실습에서는 다음을 수행합니다:

- Bedrock AgentCore Browser 기능에 대해 학습
- Strands Agents와 브라우저 자동화 테스트
- 웹사이트 탐색 및 프로그래밍 방식으로 정보 추출
- 브라우저 자동화의 일반적인 사용 사례 탐색
- 웹 상호작용 모범 사례 이해


## 사전 요구사항

이 실습을 시작하기 전에 다음 사항을 확인하세요:

- AWS 자격 증명 구성 (IAM 역할 또는 환경 변수)
- 필요한 Python 패키지 설치
- AWS 리전에 따른 Nova Pro 모델 ID

IAM 역할이 가정된 환경에서 실행하지 않는 경우, AWS 자격 증명을 환경 변수로 설정하세요:


In [ ]:
# %env AWS_REGION=<Bedrock AgentCore Region>
# %env AWS_ACCESS_KEY_ID=<YOUR ACCESS KEY>
# %env AWS_SECRET_ACCESS_KEY=<YOUR SECRET KEY>
# %env AWS_SESSION_TOKEN=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
# %env AWS_PROFILE=<OPTIONAL - Your profile configured in ~/.aws/config>

Strands Agents, Playwright 및 Bedrock AgentCore Python SDK에 필요한 패키지를 설치하세요:


In [ ]:
#%pip install -q strands-agents 'strands-agents-tools[agent_core_browser]' bedrock-agentcore playwright rich

AWS 리전에 따라 Nova Pro 모델 ID를 설정하세요:

In [ ]:
import boto3

region = boto3.session.Session().region_name

NOVA_PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"

if region.startswith("eu"):
    NOVA_PRO_MODEL_ID = "eu.amazon.nova-pro-v1:0"
elif region.startswith("ap"):
    NOVA_PRO_MODEL_ID = "apac.amazon.nova-pro-v1:0"

print(f"Nova Pro Model ID: {NOVA_PRO_MODEL_ID}")

## Bedrock AgentCore Browser란 무엇인가요?

Amazon Bedrock AgentCore Browser는 AI 에이전트가 보안 관리 환경에서 웹 브라우저와 동적으로 상호작용할 수 있게 해주는 강력한 도구입니다. 주요 기능은 다음과 같습니다:

- **웹 탐색**: 웹사이트 탐색, 요소 클릭, 프로그래밍 방식으로 양식 작성
- **콘텐츠 추출**: 웹 페이지에서 정보 추출 및 스크린샷 캡처
- **보안 환경**: 격리된 보안 브라우저 환경에서 실행
- **JavaScript 실행**: 고급 웹 상호작용을 위한 커스텀 JavaScript 실행
- **세션 관리**: 여러 작업에 걸쳐 브라우저 세션 유지

Browser 도구는 에이전트가 시각적 이해와 상호작용 기능이 필요한 복잡한 웹 기반 작업을 수행할 수 있게 합니다.


### 브라우저 세션 액세스

먼저 에이전트 환경 없이 AgentCore Browser 도구를 탐색해보겠습니다. 

Playwright 라이브러리를 사용하여 원격 브라우저 세션에 연결하겠습니다.

연결되면 실제 사용자처럼 페이지 요소와 상호작용을 시작할 수 있습니다.


In [ ]:
import boto3
import rich
from bedrock_agentcore.tools.browser_client import browser_session
from playwright.async_api import async_playwright
from IPython.display import display_jpeg

console = rich.get_console()
region = boto3.Session().region_name or "us-east-1"

with browser_session(region) as client:
    console.print(f"🌐 AgentCore Browser 세션 사용 중: {client.session_id}", style="cyan")
    ws_url, headers = client.generate_ws_headers()

    async with async_playwright() as playwright:
        chromium = playwright.chromium
        browser = await chromium.connect_over_cdp(endpoint_url=ws_url, headers=headers)
        console.print("[green]✅ 브라우저가 연결되었습니다.[/green]")

        context = browser.contexts[0] if browser.contexts else await browser.new_context()
        page = context.pages[0] if context.pages else await context.new_page()

        console.print("[cyan]🔄 AWS Builder Center로 이동 중[/cyan]")
        await page.goto("https://builder.aws.com/")
        await page.wait_for_timeout(5000)
        display_jpeg(await page.screenshot(), raw=True)

        console.print("[cyan]🔄 스포트라이트의 첫 번째 기사 클릭 중[/cyan]")
        await page.click(".swiper-wrapper div a")
        await page.wait_for_timeout(5000)
        display_jpeg(await page.screenshot(), raw=True)


### 브라우저 자동화를 사용한 Strands Agent 테스트

웹 자동화를 위해 AgentCore Browser를 사용하는 Strands Agent를 시연해보겠습니다. 

Agent 가 [Amazon Builders](https://builder.aws.com/)으로 이동하여 '/content'로 시작하는 경로로 이동하는 첫 번째 링크를 찾아 클릭하고, 페이지가 로드되면 페이지 HTML을 가져와서 기사를 요약해보도록 하겠습니다.

아래에서 오류가 발생하면 Agent 는 LLM 을 이용하여 자율적으로 문제를 해결하려고 시도합니다.


In [ ]:
import boto3
import rich
from strands import Agent
from strands.models import BedrockModel
from strands_tools.browser import AgentCoreBrowser


console = rich.get_console()
region = boto3.Session().region_name or "us-east-1"

agentcore_browser = AgentCoreBrowser(region=region)

agent = Agent(
    model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
    tools=[agentcore_browser.browser],
)

response = await agent.invoke_async("https://builder.aws.com/learn/topics/amazon-bedrock-agentcore로 이동하여 페이지가 로드되면 '/content'로 시작하는 경로로 이동하는 첫 번째 링크를 찾아 클릭하고, 페이지가 로드되면 페이지 HTML을 가져와서 기사를 요약해주세요")

console.print(response.message["content"][0]["text"] if "text" in response.message["content"][0] else "")


에이전트가 요청을 처리하고 응답을 생성하는 방법을 이해하기 위해 에이전트 루프의 상세한 실행 흐름을 살펴보겠습니다:


In [ ]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("에이전트 루프 세부사항")
console.rule()
console.print(f"루프 횟수: {agent.event_loop_metrics.cycle_count}")

table = Table(title="에이전트 메시지", show_lines=True)
table.add_column("역할", style="green")
table.add_column("텍스트", style="magenta")
table.add_column("도구 이름", style="cyan")
table.add_column("도구 입력", style="cyan")
table.add_column("도구 결과", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content]
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

### 리소스 정리

In [ ]:
client = boto3.client("bedrock-agentcore")

args = {"browserIdentifier": "aws.browser.v1", "status": "READY"}
while True:
    response = client.list_browser_sessions(**args)
    for session in response["items"]:
        client.stop_browser_session(
            browserIdentifier=args["browserIdentifier"], sessionId=session["sessionId"]
        )

    if "NextToken" in response:
        args["NextToken"] = response["NextToken"]
    else:
        break

## Bedrock AgentCore Browser의 일반적인 사용 사례

웹 검색 외에도 Bedrock AgentCore Browser는 다양한 자동화 시나리오를 가능하게 합니다:

### 웹 애플리케이션 테스트

- 보안 환경에서 웹 애플리케이션 테스트
- 사용자 인터페이스 및 기능 검증
- 자동화된 품질 보증 수행

### 데이터 수집 및 모니터링

- 웹사이트에서 정보 추출
- 웹사이트 변경사항 및 업데이트 모니터링
- 스크린샷 캡처 및 브라우저 세션 기록

### 비즈니스 프로세스 자동화

- 양식 제출 및 데이터 입력 자동화
- 전자상거래 거래 수행
- 온라인 리소스 및 서비스 액세스

### AI 기반 웹 상호작용

- 웹을 탐색할 수 있는 AI 에이전트 구축
- 웹 인터페이스와 지능적으로 상호작용
- 웹 기반 작업 및 워크플로우 수행

**추가 리소스:**
[Bedrock AgentCore Browser 사용 사례 예제](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-use-cases.html#browser-use-cases-examples)
